In [1]:
# !! /Users/simongenin/anaconda/envs/ML/bin/pip install pulp

# Cost function

numLFA : ???
numLF: ???
numLE: ???


In [3]:
### Cost Function


def costFunction(numLFA, numLF, numLE):
    return (numLFA * busData['LFA']['cost']) + (numLF * busData['LF']['cost']) + (numLE * busData['LE']['cost'])


In [31]:
### Initializing Data (routes and batteries)

# Bus Data
busData = {
    'LFA': {
        'cost' : 570000,
        'capacity' : 140
    },
    'LF':{
        'cost' : 390000,
        'capacity' : 92
    },
    'LE': {
        'cost' : 350000,
        'capacity' : 60
    }
}

# Route Data
data = {
    'timeslot_1': {
        'time': 240,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 2000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        },
        'route_7': {
            'passengers' : 3300,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        }

    },
    'timeslot_2': {
        'time': 360,
        
        'route_1' : {
            'passengers' : 700,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 30
        },
        
        'route_2' : {
            'passengers' : 300,
            'bus_frequency' : 20,
            'distance' : 10,
            'duration' : 40
        },
        'route_7': {
            'passengers' : 1000,
            'bus_frequency' : 10,
            'distance' : 10,
            'duration' : 40
        }
    }
}

# Battery Data

battData = {
    'HP': {
        'energyDensity': 60,
        'operatingV': 600,
        'maxChargeCur': 625,
        'maxDischargeCur': 500,
        'cycleLife': 6500
    },
    'MP':{
        'energyDensity': 7,
        'operatingV': 600,
        'maxChargeCur': 450,
        'maxDischargeCur': 450,
        'cycleLife': 3000
    }
}

chargeData = {
    'A': {
        'chargePower': 30,
        'chargeCur': 60
    },
    'B': {
        'chargePower': 250,
        'chargeCur': 450
    },
    'C': {
        'chargePower': 450,
        'chargeCur': 900
    }
}


In [30]:
### Importing Modules

import pandas
import pulp


In [6]:
### Calculating max appearance from timeslot, route

# def maxAppearanceLFA(route,timeslot,numLFA):
#     timeslotDuration = data[timeslot]['time']
#     roundtripDuration = data[timeslot][route]['duration']
    
#     maxAppearLFA = numLFA*timeslotDuration/(roundtripDuration+
#     return maxAppearLFA

# def maxAppearanceLF(route,timeslot,numLF):
#     timeslotDuration = data[timeslot]['time']
#     roundtripDuration = data[timeslot][route]['duration']
    
#     maxAppearLF = numLF*timeslotDuration/roundtripDuration
#     return maxAppearLF

# def maxAppearanceLE(route,timeslot,numLE):
#     timeslotDuration = data[timeslot]['time']
#     roundtripDuration = data[timeslot][route]['duration']
    
#     maxAppearLE = numLE*timeslotDuration/roundtripDuration
#     return maxAppearLE
                                            
                                            
def maxAppearanceLFA(route,timeslot,numLFA,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    maxAppearLFA = numLFA*timeslotDuration/(roundtripDuration+
    return maxAppearLFA

def maxAppearanceLF(route,timeslot,numLF,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    maxAppearLF = numLF*timeslotDuration/roundtripDuration
    return maxAppearLF

def maxAppearanceLE(route,timeslot,numLE,battType,chargeType):
    timeslotDuration = data[timeslot]['time']
    roundtripDuration = data[timeslot][route]['duration']
    
    maxAppearLE = numLE*timeslotDuration/roundtripDuration
    return maxAppearLE

# Main Algorithm

optimization of the number of buses (cost of the buses)

constraint1 : The cheapest bus should appear as much as possible
constraint2 : The buses should have enough capacity to carry the predicted number of people

In [7]:
data[timeslot]['time'] += 

SyntaxError: invalid syntax (<ipython-input-7-a4e8631d27a8>, line 1)

In [17]:
### Linear Programming Optimization

def solveLP(route,timeslot):

    # Instantiate Problem
    model = pulp.LpProblem("Cost Optimisation", pulp.LpMinimize)

    # Number of buses of types
    numLFA = pulp.LpVariable('numLFA',lowBound = 0, cat = 'Integer')
    numLF = pulp.LpVariable('numLF',lowBound = 0, cat = 'Integer')
    numLE = pulp.LpVariable('numLE',lowBound = 0, cat = 'Integer')
    
    appLFA = pulp.LpVariable('appLFA',lowBound = 0, cat = 'Integer')
    appLF = pulp.LpVariable('appLF',lowBound = 0, cat = 'Integer')
    appLE = pulp.LpVariable('appLE',lowBound = 0, cat = 'Integer')

    # Objective function
    model += costFunction(numLFA,numLF,numLE), "Cost"

    # Constraint 1 (Number of bus appearances)
    model += appLFA + appLF + appLE == data[timeslot]['time']/data[timeslot][route]['bus_frequency']
    
    model += appLFA <= maxAppearanceLFA(route,timeslot,numLFA)
    model += appLF <= maxAppearanceLF(route,timeslot,numLF)
    model += appLE <= maxAppearanceLE(route,timeslot,numLE)
    
    # Constraint 2 (Enough Passenger Capacity)

    model += (appLFA*busData['LFA']['capacity']+
                appLF*busData['LF']['capacity']+
                appLE*busData['LE']['capacity']) >= data[timeslot][route]['passengers']
    
    # Constraint 3 (Charging)
    
    
    
    # Solving
    
    model.solve()
    
    print('For ' + timeslot + ", " + route + ":")
    
    print('Status: ' + str(pulp.LpStatus[model.status]))
    
    print('numLFA: {}'.format(numLFA.varValue))
    print('numLF: {}'.format(numLF.varValue))
    print('numLE: {}'.format(numLE.varValue))
    
    print('appLFA: {}'.format(appLFA.varValue))
    print('appLF: {}'.format(appLF.varValue))
    print('appLE: {}'.format(appLE.varValue))
    
    print('Cost: ' + str(pulp.value(model.objective)))
    print('\n')

# Initializing Problems

Initialize Across Different Routes and Timeslots

In [18]:
listRoutes = ['route_1','route_2','route_3','route_4','route_5','route_6','route_7','route_8','route_9','route_10','route_11',
             'route_12','route_13']
listTimeslots = ['timeslot_1','timeslot_2','timeslot_3','timeslot_4']


solveLP('route_1', 'timeslot_1')
solveLP('route_2', 'timeslot_1')
solveLP('route_7', 'timeslot_1')

solveLP('route_1', 'timeslot_2')
solveLP('route_2', 'timeslot_2')
solveLP('route_7', 'timeslot_2')

For timeslot_1, route_1:
Status: Optimal
numLFA: 0.0
numLF: 0.0
numLE: 3.0
appLFA: 0.0
appLF: 0.0
appLE: 24.0
Cost: 1050000.0


For timeslot_1, route_2:
Status: Optimal
numLFA: 0.0
numLF: 3.0
numLE: 1.0
appLFA: 0.0
appLF: 18.0
appLE: 6.0
Cost: 1520000.0


For timeslot_1, route_7:
Status: Optimal
numLFA: 4.0
numLF: 0.0
numLE: 0.0
appLFA: 24.0
appLF: 0.0
appLE: 0.0
Cost: 2280000.0


For timeslot_2, route_1:
Status: Optimal
numLFA: 0.0
numLF: 0.0
numLE: 3.0
appLFA: 0.0
appLF: 0.0
appLE: 36.0
Cost: 1050000.0


For timeslot_2, route_2:
Status: Optimal
numLFA: 0.0
numLF: 0.0
numLE: 2.0
appLFA: 0.0
appLF: 0.0
appLE: 18.0
Cost: 700000.0


For timeslot_2, route_7:
Status: Optimal
numLFA: 0.0
numLF: 0.0
numLE: 4.0
appLFA: 0.0
appLF: 0.0
appLE: 36.0
Cost: 1400000.0


